In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# Initialize WebDriver
driver = webdriver.Chrome()
driver.get("https://psmsspa-northcentral.azurewebsites.net/#/en-US/dubai/spa-reservations")
driver.maximize_window()

# List to store service details
service_data = []

try:
    # ✅ Step 1: Wait for the dropdown to appear
    dropdown = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.ID, "userTreatments-0"))
    )
    print("✅ Dropdown found!")

    # ✅ Step 2: Click the label inside dropdown to open it
    label = WebDriverWait(dropdown, 20).until(
        EC.visibility_of_element_located((By.XPATH, ".//div[contains(@class, 'ng-value')]"))
    )
    driver.execute_script("arguments[0].scrollIntoView();", label)
    time.sleep(3)
    label.click()
    print("✅ Clicked dropdown label.")

    # ✅ Step 3: Extract all options from the dropdown
    options = WebDriverWait(driver, 20).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "ng-dropdown-panel .ng-option"))
    )
    option_texts = [option.text.strip() for option in options]
    print(f"✅ Available options: {option_texts}")
    
    # ✅ Step 4: Loop through each option and extract services
    for option_text in option_texts:
        try:
            # Re-click the dropdown label to open it
            
            label.click()
            time.sleep(2)

            # Select the current option
            option_xpath = f"//ng-dropdown-panel//span[normalize-space()='{option_text}']"
            option = WebDriverWait(driver, 20).until(
                EC.element_to_be_clickable((By.XPATH, option_xpath))
            )
            option.click()
            print(f"✅ Selected: {option_text}")
            time.sleep(2)  # Wait for services to load

            # ✅ Step 5: Extract all services under the selected category
            services = driver.find_elements(By.CSS_SELECTOR, ".panel-group .panel.panel-default")

            for service in services:
                try:
                    # Extract service name
                    name = service.find_element(By.CSS_SELECTOR, ".panel-title").text.strip()

                    # Extract description
                    description = service.find_element(By.CSS_SELECTOR, ".panel-body .col-md-6.col-sm-6").text.strip()

                    # Extract duration
                    duration = service.find_element(By.CSS_SELECTOR, ".panel-body .ng-star-inserted span").text.strip()

                    # Extract price
                    price = service.find_element(By.CSS_SELECTOR, ".panel-body .d-block").text.strip()

                    # Append service details to the list
                    service_data.append({
                        "Category": option_text,
                        "Service Name": name,
                        "Description": description,
                        "Duration": duration,
                        "Price": price
                    })

                    print(f"✅ Extracted: {name} under {option_text}")
                except Exception as e:
                    print(f"❌ Error extracting service details for {option_text}: {str(e)}")
        except Exception as e:
            print(f"❌ Error selecting option {option_text}: {str(e)}")

except Exception as e:
    print(f"❌ Error: {str(e)}")

finally:
    # Close the WebDriver
    driver.quit()

    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(service_data)

    # Save the DataFrame to a CSV file
    df.to_csv("spa_services_all_categories.csv", index=False)
    print("✅ Data saved to 'spa_services_all_categories.csv'")

✅ Dropdown found!
✅ Clicked dropdown label.
✅ Available options: ['Select treatment type', 'Massage', 'Signature Therapies', 'Programmes', 'Essence of Arabia', 'Holistic Facials', 'Advanced Facials', 'Body Treatments', 'Specialties']
❌ Error selecting option Select treatment type: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6F0D202F5+28725]
	(No symbol) [0x00007FF6F0C82AE0]
	(No symbol) [0x00007FF6F0B1510A]
	(No symbol) [0x00007FF6F0B693D2]
	(No symbol) [0x00007FF6F0B695FC]
	(No symbol) [0x00007FF6F0BB3407]
	(No symbol) [0x00007FF6F0B8FFEF]
	(No symbol) [0x00007FF6F0BB0181]
	(No symbol) [0x00007FF6F0B8FD53]
	(No symbol) [0x00007FF6F0B5A0E3]
	(No symbol) [0x00007FF6F0B5B471]
	GetHandleVerifier [0x00007FF6F104F30D+3366989]
	GetHandleVerifier [0x00007FF6F10612F0+3440688]
	GetHandleVerifier [0x00007FF6F10578FD+3401277]
	GetHandleVerifier [0x00007FF6F0DEAAAB+858091]
	(No symbol) [0x00007FF6F0C8E74F]
	(No symbol) [0x00007FF6F0C8A304]
	(No symbol) [0x00007FF6F0C8A49D]
	(No symbol) [0x0

In [9]:
df

,Category,Service Name,Description,Duration,Price
0,Massage,Oriental Essence 1 hour 30 minutes,Using custom-blended oriental oils and movemen...,90 minutes,990د.إ
1,Massage,Blissful Marma 1 hour 30 minutes,"Experience effective and immediate relaxation,...",90 minutes,990د.إ
2,Massage,Deep Relief 1 hour 30 minutes,Muscle discomfort is eased and joints are stre...,90 minutes,990د.إ
3,Massage,Pre Natal Massage 1 hour 30 minutes,A personalized body massage that has been care...,90 minutes,990د.إ
4,Signature Therapies,Oriental Qi 1 Hour 30 Minutes,"A simple, effective and authentic spa experien...",90 minutes,990د.إ
5,Signature Therapies,Digital Wellness Escape 1 Hour 30 Minute,"Concentrating on the head, eyes, neck, shoulde...",90 minutes,990د.إ
6,Signature Therapies,Intelligent Movement 1 Hour 30 Minutes,The Intelligent Movement Treatment has been de...,90 minutes,990د.إ
7,Programmes,Empowering Me - Womans Journey 2hr 30min,"Celebrate your inner and outer beauty, address...",150 minutes,1875د.إ
8,Programmes,A Gentlemans Day 2hr 30min,"For the active man to the constant traveller, ...",150 minutes,1875د.إ
9,Programmes,Sleep Support Therapy 2 hours,A tailored signature program to assist sleep c...,120 minutes,1300د.إ
